# Nearest Neighbour

Nearest neightbour provides functionality for unsupervised and supervised neighbors-based learning methods. Unsupervised nearest neighbors is the foundation of many other learning methods, notably manifold learning and spectral clustering. Supervised neighbors-based learning comes in two flavors: classification for data with discrete labels, and regression for data with continuous labels.

The principle behind nearest neighbor methods is to find a predefined number of training samples closest in distance to the new point, and predict the label from these. The number of samples can be a user-defined constant (k-nearest neighbor learning), or vary based on the local density of points (radius-based neighbor learning). The distance can, in general, be any metric measure: standard Euclidean distance is the most common choice.

Being a non-parametric method, it is often successful in classification situations where the decision boundary is very irregular.

---

## Math Principle

The classification is based on the class of the nearest neighbours. I.e. the point will check the most representative nearest neighbour's class and decide the class.

A very commonly used algorithm is KNN, in which k stands for the number of neighbor the point will check.

The sklearn also provide rnn, where the algorithm will check all points's classes within the radius assigned by the user.

For knn, which is more commonly used, the k value is highly dependent to the data. Large the k value, more robust the model, but less clear the boundries. This issue can be solved with Silhouette Score and other evaluators.

For rnn, it is a better choice when data is not sampled uniformly. But for higher dimension data, its performance is poor.

By default, the data's weight is the same, user can change the weight with parameter `weights`. 

---

## Realization

1. Brute Force
   
   This algorithm is useful for small data. We can use it by setting parameter `algorithm=brute`

2. K-D Tree
   
   K-D Tree is a more efficient algorithm comparing to the brute force. The basic idea is to generate a k dimensional $2^k$ tree recursively partitions the parameter space along the data axes, dividing it into nested orthotropic regions into which data points are filed.

   Once the K-D Tree is constructed, it will be super fast to check the nearest neighbor. However, as the dimension goes up, the speed of checking nearest points become slow.

3. Ball Tree
   
   To address the inefficiencies of KD Trees in higher dimensions, the ball tree data structure was developed. Where KD trees partition data along Cartesian axes, ball trees partition data in a series of nesting hyper-spheres. This makes tree construction more costly than that of the KD tree, but results in a data structure which can be very efficient on highly structured data, even in very high dimensions.

   A ball tree recursively divides the data into nodes defined by a centroid $C$ and radius $r$ , such that each point in the node lies within the hyper-sphere defined by $r$ and $C$. 

In [2]:
# Sample Code for KDTree
# Just a demo, plz don't write it yourself

class Node:
    def __init__(self, point, left=None, right=None):
        self.point = point
        self.left = left
        self.right = right

def build_kd_tree(points, depth=0):
    if not points:
        return None

    k = len(points[0])
    axis = depth % k
    points.sort(key=lambda point: point[axis])
    median_idx = len(points) // 2
    median = points[median_idx]
    
    left_points = points[:median_idx]
    right_points = points[median_idx + 1:]
    
    left_subtree = build_kd_tree(left_points, depth + 1)
    right_subtree = build_kd_tree(right_points, depth + 1)
    
    return Node(median, left_subtree, right_subtree)

def print_tree(node, level=0):
    if node:
        print(level, node.point)
        print_tree(node.left, level + 1)
        print_tree(node.right, level + 1)

data = [(2,3,4), (5,4,6), (9,6,8), (4,7,2), (8,1,5), (7,2,3)]
root = build_kd_tree(data)

print_tree(root)

0 (7, 2, 3)
1 (5, 4, 6)
2 (2, 3, 4)
2 (4, 7, 2)
1 (9, 6, 8)
2 (8, 1, 5)


In [2]:
# Sample Code for Ball Tree
# Just a demo, plz don't write it yourself
import numpy as np

class BallNode:
    def __init__(self, center, radius, left=None, right=None, points=None):
        self.center = center  # Ball 的球心
        self.radius = radius  # Ball 的半径
        self.left = left  # 左子树
        self.right = right  # 右子树
        self.points = points  # 子树中的数据点

def build_ball_tree(points):
    if len(points) == 0:
        return None

    center = np.mean(points, axis=0)  # 计算球心
    radius = np.max(np.linalg.norm(points - center, axis=1))  # 计算半径

    if len(points) <= leaf_size:  # 叶节点
        return BallNode(center, radius, points=points)

    left_indices = np.random.choice(len(points), len(points) // 2, replace=False)
    left_points = points[left_indices]
    right_points = np.delete(points, left_indices, axis=0)

    left_child = build_ball_tree(left_points)
    right_child = build_ball_tree(right_points)

    return BallNode(center, radius, left=left_child, right=right_child)

def print_ball_tree(node, depth=0):
    if node is None:
        return

    print("  " * depth, f"Center: {node.center}, Radius: {node.radius}")

    if node.points is not None:
        print("  " * (depth + 1), "Leaf Points:", node.points)
    else:
        print("  " * (depth + 1), "Left Child:")
        print_ball_tree(node.left, depth + 2)
        print("  " * (depth + 1), "Right Child:")
        print_ball_tree(node.right, depth + 2)

np.random.seed(0)
points = np.random.rand(100, 2)  # 生成随机数据点

global leaf_size
leaf_size = 5  # 叶节点中允许的数据点数量

ball_tree = build_ball_tree(points)
print("Ball Tree construction completed.")
print("Printing Ball Tree Structure:")
print_ball_tree(ball_tree)

Ball Tree construction completed.
Printing Ball Tree Structure:
 Center: [0.51396978 0.48690581], Radius: 0.6108823126671181
   Left Child:
     Center: [0.49451337 0.47757896], Radius: 0.6204224830631309
       Left Child:
         Center: [0.55003075 0.50610432], Radius: 0.5961616807725127
           Left Child:
             Center: [0.62264105 0.41348682], Radius: 0.6872600527948831
               Left Child:
                 Center: [0.59830465 0.52933042], Radius: 0.5848895966927243
                   Left Child:
                     Center: [0.84605675 0.58723271], Radius: 0.3590770132264817
                       Leaf Points: [[0.97861834 0.79915856]
 [0.88173536 0.69253159]
 [0.67781654 0.27000797]]
                   Right Child:
                     Center: [0.35055254 0.47142813], Radius: 0.5006898539724937
                       Leaf Points: [[0.72999056 0.17162968]
 [0.18619301 0.94437239]
 [0.13547406 0.29828233]]
               Right Child:
                 Center: [0.64

## Method, Parameter and Attribute

